# Update Program Code Descriptions

In [1]:
import numpy as np
import pandas as pd
from siuba import *

from shared_utils import geography_utils
from dla_utils import _dla_utils

from calitp import to_snakecase


import utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

In [51]:
df = utils.read_data_all()
    
    ## function that adds known agency name to df 
df = utils.identify_agency(df, 'summary_recipient_defined_text_field_1_value')

In [52]:
df.sample(3)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
248,2022-03-24,Y001,National Highway Performance Program (NHPP),5275021,08925183L,"WESTBOUND INDIO BOULEVARD OVER WHITEWATER RIVER, BR. NO. 56C-0292 SEISMIC RETROFIT AND SCOUR COUNTERMEASURES",65.00,Cong Dist 36,14.00,Bridge Rehabilitation - No Added Capacity,"1,190,374.00",L5275SCAG,"5,275.00",Indio,8.00,Riverside County,Riverside County Transportation Commission,Southern California Association Of Governments
1156,2022-05-10,Y001,National Highway Performance Program (NHPP),0081328,1119000177S,IN SAN DIEGO COUNTY IN EL CAJON AT VARIOUS LOCATIONS FROM 0.5 MILE EAST OF GROSSMONT BOULEVARD OVERCROSSING TO 0.4 MILE EAST OF LOS COCHES ROAD UNDER,73.00,Cong Dist 50,6.00,4R - Restoration & Rehabilitation,"2,380,971.90",S SANDAG,NaN,California,11.00,San Diego County,NaN,NaN
407,2022-09-19,YS30,Highway Safety Improvement Program (HSIP),5930094,1019000151L,"SOUTHWORTH RD, GOLD STRIKE RD, POOL STATION RD, GARABALDI ST, MURPHYS GRADE RD, COPPER COVE DR, AND O'BYRNES FERRY RD. UPGRADE EXISTING GUARDRAILS,",9.00,Cong Dist 4,17.00,Construction Engineering,"36,765.00",L5930NON-MPO,"5,930.00",Calaveras County,10.00,Calaveras County,Calaveras Council of Governments,NON-MPO


In [5]:
#checking to make sure summary_recipient_defined_text_field_1_value (and implementing_agency) is unique
df>>group_by(_.project_number)>>summarize(n = _.summary_recipient_defined_text_field_1_value.nunique())>>arrange(-_.n)

,project_number,n
0,000C480,1
1,000C524,1
2,000C529,1
3,000C530,1
4,000C531,1
...,...,...
547,X075048,1
548,X077028,1
549,X081034,1
550,X093010,1


## Condense Projects into One

In [6]:
df_agg = df.astype(str).groupby('project_number').agg(' | ' .join).reset_index()

In [7]:
df_agg.sample()

,project_number,fmis_transaction_date,program_code,program_code_description,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
458,P055064,2022-09-13 | 2022-09-13 | 2022-09-13 | 2022-09-13 | 2022-09-13 | 2022-09-13,YS30 | YS30 | YS30 | YS30 | YS30 | YS30,Highway Safety Improvement Program (HSIP) | Highway Safety Improvement Program (HSIP) | Highway Safety Improvement Program (HSIP) | Highway Safety Improvement Program (HSIP) | Highway Safety Improvement Program (HSIP) | Highway Safety Improvement Program (HSIP),1219000083S | 1219000083S | 1219000083S | 1219000083S | 1219000083S | 1219000083S,ORANGE AND ANAHEIM AT VARIOUS LOCATIONS FROM 0.1 MILE SOUTH OF KATELLA AVENUE UNDERCROSSING TO 0.5 MILE NORTH OF LINCOLN AVENUE UNDERCROSSING UPGRADE | ORANGE AND ANAHEIM AT VARIOUS LOCATIONS FROM 0.1 MILE SOUTH OF KATELLA AVENUE UNDERCROSSING TO 0.5 MILE NORTH OF LINCOLN AVENUE UNDERCROSSING UPGRADE | ORANGE AND ANAHEIM AT VARIOUS LOCATIONS FROM 0.1 MILE SOUTH OF KATELLA AVENUE UNDERCROSSING TO 0.5 MILE NORTH OF LINCOLN AVENUE UNDERCROSSING UPGRADE | ORANGE AND ANAHEIM AT VARIOUS LOCATIONS FROM 0.1 MILE SOUTH OF KATELLA AVENUE UNDERCROSSING TO 0.5 MILE NORTH OF LINCOLN AVENUE UNDERCROSSING UPGRADE | ORANGE AND ANAHEIM AT VARIOUS LOCATIONS FROM 0.1 MILE SOUTH OF KATELLA AVENUE UNDERCROSSING TO 0.5 MILE NORTH OF LINCOLN AVENUE UNDERCROSSING UPGRADE | ORANGE AND ANAHEIM AT VARIOUS LOCATIONS FROM 0.1 MILE SOUTH OF KATELLA AVENUE UNDERCROSSING TO 0.5 MILE NORTH OF LINCOLN AVENUE UNDERCROSSING UPGRADE,59.0 | 59.0 | 59.0 | 59.0 | 59.0 | 59.0,Cong Dist 39 | Cong Dist 39 | Cong Dist 45 | Cong Dist 45 | Cong Dist 46 | Cong Dist 46,17.0 | 21.0 | 17.0 | 21.0 | 17.0 | 21.0,Construction Engineering | Safety | Construction Engineering | Safety | Construction Engineering | Safety,24639.07 | 54707.86 | 197903.72 | 439419.57 | 263457.21 | 584972.57,S SCAG | S SCAG | S SCAG | S SCAG | S SCAG | S SCAG,nan | nan | nan | nan | nan | nan,California | California | California | California | California | California,12.0 | 12.0 | 12.0 | 12.0 | 12.0 | 12.0,Orange County | Orange County | Orange County | Orange County | Orange County | Orange County,nan | nan | nan | nan | nan | nan,nan | nan | nan | nan | nan | nan


In [8]:
(list(df.columns))

['fmis_transaction_date',
 'program_code',
 'program_code_description',
 'project_number',
 'recipient_project_number',
 'project_title',
 'county_code',
 'congressional_district',
 'improvement_type',
 'improvement_type_description',
 'obligations_amount',
 'summary_recipient_defined_text_field_1_value',
 'implementing_agency_locode',
 'implementing_agency',
 'district',
 'county_name',
 'rtpa_name',
 'mpo_name']

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1241 entries, 0 to 1211
Data columns (total 18 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fmis_transaction_date                         1241 non-null   datetime64[ns]
 1   program_code                                  1241 non-null   object        
 2   program_code_description                      1240 non-null   object        
 3   project_number                                1241 non-null   object        
 4   recipient_project_number                      1240 non-null   object        
 5   project_title                                 1241 non-null   object        
 6   county_code                                   1241 non-null   float64       
 7   congressional_district                        1241 non-null   object        
 8   improvement_type                              1241 non-null   float6

In [10]:
df2 = df.copy()

In [11]:
df_agg = utils.condense_df(df2)

In [12]:
df_agg.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name
50,2022-03-07,5906119,Shasta County,L5906SHASTA,Y001,National Highway Performance Program (NHPP),0214000079L,11.0 | 17.0,Bridge Replacement - No Added Capacity | Construction Engineering,"GAS POINT ROAD BRIDGE (06C-0183) OVER NO NAME DITCH, 1.9 MILES WEST OF I-5. BR.# 06C0183 BRIDGE REPLACEMENT","831,795.00",Cong Dist 1,2.0,89.0,Shasta County,5906.0,Shasta County Regional Transportation Planning Agency,Shasta County Regional Transportation Planning Age
478,2022-09-12,P204006,California,S KCOG,YS30,Highway Safety Improvement Program (HSIP),0617000297S,17.0 | 21.0,Construction Engineering | Safety,KERN COUNTY IN BAKERSFIELD AT 8TH STREET. INSTALLATION OF HIGH INTENSITY ACTIVATED CROSSWALK.,"1,818,500.00",Cong Dist 21,9.0,29.0,Kern County,nan,None,None
31,2022-02-17,5951141,Santa Barbara County,L5951SBCAG,Y233,Surface Transportation Block Grant (STBG) Program,0500020352L,11.0 | 15.0 | 17.0,Bridge Replacement - No Added Capacity | Preliminary Engineering | Construction Engineering,FERNALD PT. LN. BRIDGE #51C0137 @ ROMERO CRK IN SANTA BARBARA COUNTY BRIDGE REPLACEMENT (TC),"1,175,253.00",Cong Dist 24,5.0,83.0,Santa Barbara County,5951.0,Santa Barbara County Association of Governments,Santa Barbara County Association Of Governments
439,2022-09-08,0052036,California,S SCAG,YS30,Highway Safety Improvement Program (HSIP),1218000102S,17.0 | 21.0,Construction Engineering | Safety,"IN ORANGE COUNTY IN ANAHEIM AT KATELLA AVENUE UNDERCROSSING COLD PLANE, PLACE RHMA & HSFT, REPLACE SAFETY DEVICES AND SIGNS","1,649,400.00",Cong Dist 46,12.0,59.0,Orange County,nan,None,None
399,2022-08-30,X093010,California,S NON-MPO,Y237,Surface Transportation Block Grant (STBG) Program,0217000009S,6.0,4R - Restoration & Rehabilitation,IN SISKIYOU COUNTY IN YREKA ON ROUTE 3 FROM 0.4 MILE NORTH OF LAURA LANE TO JUNIPER DRIVE AND ON ROUTE 263 FROM ROUTE 3 TO 1.0 MILE SOUTH OF LONG GUL,"24,311,937.39",Cong Dist 1,2.0,93.0,Siskiyou County,nan,None,None


In [13]:
#should return 552
len(df_agg)

552

In [14]:
# def condense_df(df):
    
#     # make sure columns are in string format
#     df[['county_code', 'improvement_type',
#      'implementing_agency_locode', 'district',
#      'program_code_description', 'recipient_project_number']] = df[['county_code', 'improvement_type',
#                                                                      'implementing_agency_locode', 'district',
#                                                                      'program_code_description', 'recipient_project_number']].astype(str)
#     # aggreate df using .agg function and join in the unique values into one row
#     df_agg = (df
#            .assign(count=1)
#            .groupby(['fmis_transaction_date','project_number', 'implementing_agency', 'summary_recipient_defined_text_field_1_value'])
#            .agg({'program_code':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'program_code_description':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'recipient_project_number':lambda x:' | '.join(x.unique()), #'first',
#                  'improvement_type':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'improvement_type_description':lambda x:' | '.join(x.unique()),  # get unique values to concatenate
#                  'project_title':'first', #should be the same                 
#                  'obligations_amount':'sum', #sum of the obligations amount
#                  'congressional_district':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'district':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'county_code':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'county_name':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'implementing_agency_locode':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'rtpa_name':'first', #should be the same
#                  'mpo_name':'first',  #should be the same
#                 }).reset_index())
    
#     return df_agg

In [15]:
# df2[['county_code'
#      , 'improvement_type',
#      'implementing_agency_locode',
#      'district',
#      'program_code_description',
#      'recipient_project_number']] = df2[['county_code',
#                                          'improvement_type',
#                                          'implementing_agency_locode',
#                                          'district', 'program_code_description',
#                                          'recipient_project_number']].astype(str)

In [16]:
# df_agg = (df2
#            .assign(count=1)
#            .groupby(['fmis_transaction_date','project_number', 'implementing_agency', 'summary_recipient_defined_text_field_1_value'])
#            .agg({'program_code':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'program_code_description':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'recipient_project_number':lambda x:' | '.join(x.unique()), #'first',
#                  'improvement_type':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'improvement_type_description':lambda x:' | '.join(x.unique()),  # get unique values to concatenate
#                  'project_title':'first', #should be the same                 
#                  'obligations_amount':'sum', #sum of the obligations amount
#                  'congressional_district':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'district':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'county_code':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'county_name':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'implementing_agency_locode':lambda x:' | '.join(x.unique()), # get unique values to concatenate
#                  'rtpa_name':'first', #should be the same
#                  'mpo_name':'first',  #should be the same
#                 }).reset_index())

In [17]:
len(df_agg>>filter(_.improvement_type.str.contains(' | ')))

320

In [19]:
full_df_agg = utils.get_new_desc_title(df_agg)

In [ ]:
full_df_agg

## New issue: `project_title_new` taking groupings from `improvement_type_description`


In [ ]:
#checking the description column to add "project" flag and subsetting data
test_df = (utils.add_description(full_df_agg, 'project_title'))>>filter(_.project_type=='Project')
# test_df = (test_df>>select(_.implementing_agency, _.program_code_description, _.improvement_type, _.improvement_type_description, _.project_title,
#                           _.county_name, _.project_title_new, _.project_type))

In [ ]:
len(test_df)

In [ ]:
print((f"There are {len(test_df>>filter(_.improvement_type.str.contains(' | ')))} "
           f"entires out of {len(full_df_agg)} that have concatenated entired for improvement_type_description"))

In [ ]:
#entries with concatenated improvement types descriptions
(test_df>>filter(_.improvement_type.str.contains(' | '))).improvement_type_description.value_counts()

In [ ]:
## filter down to just the projects with multiple improvement type descriptions
test_df = (test_df>>filter(_.improvement_type.str.contains(' | ')))

In [ ]:
# #function to rework DOES NOT WORK
# def update_no_matched(df, flag_col, desc_col, program_code_desc_col): 
#     """
#     function to itreate over projects that did not match the first time
#     using an existing project's short description of project type. 
#     """
    
#     def return_project_type(df):
        
#         if (df[flag_col] == "Project") & df[desc_col].str.contains("Bridge Rehabilitation") | df[desc_col].str.contains("Bridge Rehabilitation - No Added Capacity") | df[desc_col].str.contains("Bridge Rehabilitation - Added Capacity"):
#             return ("Bridge Rehabilitation")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Facilities for Pedestrians and Bicycles"):
#             return (df[desc_col])
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Safety"):
#             return (df[desc_col] + " Improvements")
            
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Planning "):
#             return "Project Planning" 
            
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Preliminary Engineering"):
#             return (df[desc_col] + " Projects ")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Construction Engineering"):
#             return (df[desc_col] + " Projects")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("4R - Restoration & Rehabilitation"):
#             return ("Road Restoration & Rehabilitation")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("4R - Maintenance  Resurfacing"):
#             return ("Maintenance Resurfacing")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Bridge Replacement - Added Capacity")| df[desc_col].str.contains("Bridge Replacement - No Added Capacity") | df[desc_col].str.contains("Bridge New Construction")| df[desc_col].str.contains("Special Bridge"):
#             return ("Bridge Replacement")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Mitigation of Water Pollution due to Highway Runoff"):
#             return (df[desc_col])
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("4R - Added Capacity"):
#             return ("Added Roadway Capacity")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("4R - No Added Capacity"):
#             return ("Road Construction")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("New  Construction Roadway"):
#             return ("New Construction Roadway")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Traffic Management/Engineering - HOV"):
#             return ("Traffic Management Project")
        
#         elif (df[flag_col] == "Project") & df[desc_col].str.contains("Right of Way"):
#             return (df[desc_col] + " Project")
        
#         # elif (df[flag_col] == "Project") & (df[program_code_desc_col]== "National Highway Performance Program (NHPP)"): 
#         #     return ("National Highway Performance Program Support") 
        
#         # elif (df[flag_col] == "Project") & (df[desc_col] != "Other"):
#         #     return (df[desc_col])
    
#         else:
#             return df[flag_col] 

#         return df

#     df['project_type2'] = df.apply(return_project_type, axis = 1)

#     return df

In [ ]:
#test_df2 = update_no_matched(test_df, 'project_type', 'improvement_type_description', 'program_code_description')

In [ ]:
#test_df2.sample(5)

In [ ]:
#trying another version
#update: worked-- moving to utils
def add_description_4_no_match(df, desc_col):
    ##using np.where. code help: https://stackoverflow.com/questions/43905930/conditional-if-statement-if-value-in-row-contains-string-set-another-column

    
    ## method for project in first column
    df['project_type2'] = (np.where(df[desc_col].str.contains("Bridge Rehabilitation"),"Bridge Rehabilitation",
                        np.where(df[desc_col].str.contains("Bridge Rehabilitation - No Added Capacity") | df[desc_col].str.contains("Bridge Rehabilitation - Added Capacity"), "Bridge Rehabilitation",
                        np.where(df[desc_col].str.contains("Bridge Replacement - Added Capacity")| df[desc_col].str.contains("Bridge Replacement - No Added Capacity"), "Bridge Replacement",
                        np.where(df[desc_col].str.contains("Bridge New Construction")| df[desc_col].str.contains("Special Bridge"), "Bridge Replacement",
                        np.where(df[desc_col].str.contains("Facilities for Pedestrians and Bicycles"), "Facilities for Pedestrians and Bicycles",
                        np.where(df[desc_col].str.contains("Mitigation of Water Pollution due to Highway Runoff"), "Mitigation of Water Pollution due to Highway Runoff",
                        np.where(df[desc_col].str.contains("Traffic Management/Engineering - HOV"), "Traffic Management Project",
                        np.where(df[desc_col].str.contains("Planning "), "Project Planning",
                        np.where(df[desc_col].str.contains("4R - Restoration & Rehabilitation"), "Road Restoration & Rehabilitation",
                        np.where(df[desc_col].str.contains("4R - Maintenance  Resurfacing"), "Maintenance Resurfacing",
                        np.where(df[desc_col].str.contains("4R - Added Capacity"), "Added Roadway Capacity",
                        np.where(df[desc_col].str.contains("4R - No Added Capacity"), "Road Construction",
                        np.where(df[desc_col].str.contains("Safety"), "Safety Improvements",
                        np.where(df[desc_col].str.contains("New  Construction Roadway"), "New Construction Roadway",
                        np.where(df[desc_col].str.contains("Preliminary Engineering"), "Preliminary Engineering Projects",
                        np.where(df[desc_col].str.contains("Construction Engineering"), "Construction Engineering Projects",
                        np.where(df[desc_col].str.contains("Right of Way"), "Right of Way Project",
                                    "Project"))))))))))))))))))
    
    return df

In [ ]:
test_df3 = add_description_4_no_match(test_df, 'improvement_type_description')

In [ ]:
test_df3.project_type2.value_counts()

In [ ]:
test_df3.loc[0:60]

In [ ]:
test_df3>>filter(_.improvement_type_description.str.contains("Other"))

## Add new function to larger function

In [53]:
# using df_agg

df = utils.condense_df(df)

In [54]:
proj_unique_cat = utils.add_description(df, 'project_title')

In [55]:
len(proj_unique_cat>>filter(_.project_type=='Project'))

205

In [56]:
proj_unique_cat.loc[proj_unique_cat['project_type'] == 'Project', 'project_method'] = ""

In [57]:
proj_unique_cat['project_type'] = proj_unique_cat['project_type'].replace('Project', np.NaN)

In [58]:
proj_unique_cat_title =  utils.add_description_4_no_match(proj_unique_cat, 'improvement_type_description')


In [59]:
proj_unique_cat_title['project_type'] = proj_unique_cat_title['project_type'].fillna(proj_unique_cat_title['project_type2'])


In [60]:
proj_unique_cat_title = utils.add_new_title(proj_unique_cat, "project_method", "project_type", "implementing_agency", "county_name")


In [61]:
proj_unique_cat_title = proj_unique_cat_title.rename(columns={'project_name_new':'project_title_new'})
#proj_unique_cat_title.drop(columns =['project_method', 'project_type', 'project_type2'], axis=1, inplace=True)

In [62]:
proj_title_mapping = (dict(proj_unique_cat_title[['project_number', 'project_title_new']].values))

In [64]:
df['project_title_new'] = df.project_number.map(proj_title_mapping)

In [65]:
df

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,project_method,project_type,project_type2,project_name_new,project_title_new
0,2022-02-04,5064095,Pasadena,L5064SCAG,YS30,Highway Safety Improvement Program (HSIP),0721000154L,17.0 | 21.0,Construction Engineering | Safety,VARIOUS LOCATIONS THROUGHOUT THE WEST SIDE OF THE CITY. REPAIR EXISTING DAMAGED GUARDRAI,"289,000.00",Cong Dist 27,7.0,37.0,Los Angeles County,5064.0,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments,Repair,Guardrails,Safety Improvements,Repair Guardrails in Pasadena,Repair Guardrails in Pasadena
1,2022-02-08,5002199,Sacramento,L5002SACOG,YS30,Highway Safety Improvement Program (HSIP),0317000138L,21.0 | 17.0,Safety | Construction Engineering,"INTERSECTIONS OF 65TH ST EXPWY AT 21ST AVE, 65TH ST EXPWY AT FRUITRIDGE RD, ARDEN WAY AT HERITAGE LN, ARDEN WAY AT CHALLENGE WAY, HOWE AVE. AT SWARTH","1,441,500.00",Cong Dist 3 | Cong Dist 6,3.0,67.0,Sacramento County,5002.0,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Safety Improvements,Safety Improvements,Safety Improvements in Sacramento,Safety Improvements in Sacramento
2,2022-02-08,5002209,Sacramento,L5002SACOG,YS30,Highway Safety Improvement Program (HSIP),0319000101L,17.0 | 21.0,Construction Engineering | Safety,DUCKHORN DRIVE FROM ARENA BOULEVARD TO FAR NIENTE WAY INSTALL CURVE WARNING SIGNS AND INSTALL RAISED MEDIANS.,"622,170.00",Cong Dist 6,3.0,67.0,Sacramento County,5002.0,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,Install,Signage,Safety Improvements,Install Signage in Sacramento,Install Signage in Sacramento
3,2022-02-09,5117016,Hercules,L5117MTC,Y230,Surface Transportation Block Grant (STBG) Program,0420000346L,5.0,4R - Maintenance Resurfacing,HERCULES: SYCAMORE AVE FROM CIVIC DR TO WILLOW/PALM AVE PAVEMENT REHABILITATION,"492,000.00",Cong Dist 5,4.0,13.0,Contra Costa County,5117.0,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Pave,Pavement Rehabilitation,Maintenance Resurfacing,Pave Pavement Rehabilitation in Hercules,Pave Pavement Rehabilitation in Hercules
4,2022-02-10,5005154,San Jose,L5005MTC,Y230,Surface Transportation Block Grant (STBG) Program,0419000079L,4.0 | 17.0,4R - No Added Capacity | Construction Engineering,"ON MCKEE ROAD BETWEEN ROUTE 101 AND TOYON AVENUE IMPLEMENT SAFETY IMPROVEMENTS INCLUDING MEDIAN ISLANDS, ADA CURB RAMP, SPEED RADAR SIGN, ENHANCED CR","6,994,933.00",Cong Dist 19,4.0,85.0,Santa Clara County,5005.0,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Improve,Signage,Road Construction,Improve Signage in San Jose,Improve Signage in San Jose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,2022-09-22,6507035,San Bernardino County Transportation Authority,L6507SCAG,Y230,Surface Transportation Block Grant (STBG) Program,0822000183L,15.0,Preliminary Engineering,"I-10 CORRIDOR CONTRACT 2: THE PROJECT WILL PROVIDE ONE EXPRESS LANES IN EACH DIRECTION FROM JUST EAST OF I-15 TOPEPPER AVENUE IN COLTON, CONNECTING T","14,000,000.00",Cong Dist 35,8.0,71.0,San Bernardino County,6507.0,NON-RTPA,Southern California Association Of Governments,,Express Lanes,Preliminary Engineering Projects,Express Lanes in San Bernardino County Transportation Authority,Express Lanes in San Bernardino County Transportation Authority
548,2022-09-22,P057073,Los Angeles County Metropolitan Transportation Authority,L6065SCAG,Y460,National Highway Freight Program (NHFP),0715000076S,11.0 | 17.0 | 42.0,Bridge Replacement - No Added Capacity | Construction Engineering | Training,IN LOS ANGELES COUNTY

In [66]:
# from get_new_desc_title
    
    
#     proj_unique_cat = add_description(proj_unique, 'project_title')
    
#     #remove project method column values so that the title function wont double count
#     proj_unique_cat.loc[proj_unique_cat['project_type'] == 'Project', 'project_method'] = ""
    # proj_unique_cat['project_type'].replace('Project', np.NaN)
    
    #update for the projects not in the first round of descriptions
    # proj_unique_cat_title =  add_description_4_no_match(proj_unique_cat, 'improvement_type_description')
    ## proj_unique_cat_title = update_no_matched(proj_unique_cat, "project_type", 'improvement_type_description', 'program_code_description')
    
    #fill nan values in 'Project_type' with values from 'project_type2' from add_description_4_no_match function
    # proj_unique_cat_title['project_type'] = proj_unique_cat_title['project_type'].fillna(proj_unique_cat_title['project_type2'])
    
    #add title - second round to account for statewide projects
    # proj_unique_cat_title = add_new_title(proj_unique_cat, "project_method", "project_type", "implementing_agency", "county_name")
    
    # rename new title one
    # proj_unique_cat_title = proj_unique_cat_title.rename(columns={'project_name_new':'project_title_new'})
    # proj_unique_cat_title.drop(columns =['project_method', 'project_type', 'project_type2'], axis=1, inplace=True)
    
    #map the title back to df
#     proj_title_mapping = (dict(proj_unique_cat_title[['project_number', 'project_title_new']].values))
    
#     df['project_title_new'] = df.project_number.map(proj_title_mapping)

#     return df

## Test & Export

In [15]:
#GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [16]:
#test agg. PASS df with title column, concat and 552 rows
#agg = utils.get_clean_data(full_or_agg = 'agg')

In [17]:
#agg = utils.title_column_names(agg)

In [18]:
#agg.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_agg.csv")

In [19]:
#test full. PASS title, no concat and 1241 rows
#full = utils.get_clean_data(full_or_agg = 'full')

In [20]:
#full = utils.title_column_names(full)

In [21]:
#full.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_all.csv")